In [1]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import pandas as pd
import numpy as np
import glob

In [ ]:
def read_and_extract_from_file(filepath):
    try:
        with open(filepath, "rb") as file:
            compressed_data = file.read()
            data = pickle.loads(lz4.frame.decompress(compressed_data))
            # print(f"Data from {filepath}: {data[:1]}")
            return data
    except FileNotFoundError:
        print(f"No file found: {filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return None

In [ ]:
# Process files and inspect data
def process_files_in_folder(folder_path):
    files = glob.glob(os.path.join(folder_path, "*.pickle.lz4"))
    
    df_list = []
    
    for file in files:
        symbol = os.path.basename(file).split('-')[0]
        data = read_and_extract_from_file(file)
        
        if data is None or len(data) == 0:
            continue
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Only proceed if the expected columns exist
        if {'close', 'open', 'high', 'low', 'volume', 'timestamp'}.issubset(df.columns):
            df_filtered = df[['close', 'open', 'high', 'low', 'volume', 'timestamp']].copy()
            df_filtered['symbol'] = symbol
            df_filtered = df_filtered[['symbol', 'close', 'open', 'high', 'low', 'volume', 'timestamp']]
            # Convert the 'timestamp' column from milliseconds to datetime
            df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'] / 1000, unit='s')
            # Format the 'timestamp' as 'yyyy-mm-dd'
            df_filtered['timestamp'] = df_filtered['timestamp'].dt.strftime('%Y-%m-%d')
            df_list.append(df_filtered)
        else:
            print(f"Missing expected columns in {file}. Columns available: {df.columns}")
    
    if df_list:
        full_df = pd.concat(df_list, ignore_index=True)
        return full_df
    else:
        print("No data was processed.")
        return None

In [10]:
# Main function to run the process
def main():
    folder_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\rawAggs"
    df = process_files_in_folder(folder_path)
    
    if df is not None:
        print(df.head())

if __name__ == "__main__":
    main()

Data from C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\StockTrading\Data\rawAggs\AACT-aggs-2022-01-01_to_2024-09-30.pickle.lz4: [Agg(open=10.1, high=10.15, low=10.1, close=10.12, volume=603102, vwap=10.1004, timestamp=1686542400000, transactions=27, otc=None)]
Data from C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\StockTrading\Data\rawAggs\AAOI-aggs-2022-01-01_to_2024-09-30.pickle.lz4: [Agg(open=5.2, high=5.305, low=5.08, close=5.27, volume=478578, vwap=5.1922, timestamp=1641186000000, transactions=3244, otc=None)]
Data from C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\StockTrading\Data\rawAggs\AAT-aggs-2022-01-01_to_2024-09-30.pickle.lz4: [Agg(open=37.59, high=38.1, low=37.35, close=37.66, volume=187059, vwap=37.6581, timestamp=1641186000000, transactions=3881, otc=None)]
Data from C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\StockTrading\Data\rawAggs\ABCL-aggs-2022-01-01_to_2024-09-30.pickle.lz4: [Agg(open=14.47, high=14.79, low=13.93, close=14